<a href="https://colab.research.google.com/github/mashuai191/pantograph/blob/master/Pantograph_Demo_localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip list | grep tensor

tensorboard              2.2.0          
tensorboard-plugin-wit   1.6.0.post2    
tensorboardcolab         0.0.22         
tensorflow               2.2.0rc2       
tensorflow-addons        0.8.3          
tensorflow-datasets      2.1.0          
tensorflow-estimator     2.2.0rc0       
tensorflow-gcs-config    2.1.8          
tensorflow-hub           0.7.0          
tensorflow-metadata      0.21.1         
tensorflow-privacy       0.2.2          
tensorflow-probability   0.9.0          


In [0]:
#!pip install tensorflow==1.14
#!pip install keras==2.1.5

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
!pip install flask-ngrok


In [0]:
import atexit
import json
import os
import platform
import shutil
import subprocess
import tempfile
import time
import zipfile
from pathlib import Path
from threading import Timer

import requests


def _get_command():
    system = platform.system()
    if system == "Darwin":
        command = "ngrok"
    elif system == "Windows":
        command = "ngrok.exe"
    elif system == "Linux":
        command = "ngrok"
    else:
        raise Exception("{system} is not supported".format(system=system))
    return command


def _run_ngrok(port):
    command = _get_command()
    ngrok_path = str(Path(tempfile.gettempdir(), "ngrok"))
    _download_ngrok(ngrok_path)
    executable = str(Path(ngrok_path, command))
    os.chmod(executable, 0o777)
    ngrok = subprocess.Popen([executable, 'http', str(port)])
    atexit.register(ngrok.terminate)
    localhost_url = "http://localhost:4040/api/tunnels"  # Url with tunnel details
    time.sleep(1)
    tunnel_url = requests.get(localhost_url).text  # Get the tunnel information
    j = json.loads(tunnel_url)

    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
    tunnel_url = tunnel_url.replace("https", "http")
    return tunnel_url


def _download_ngrok(ngrok_path):
    if Path(ngrok_path).exists():
        return
    system = platform.system()
    if system == "Darwin":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-darwin-amd64.zip"
    elif system == "Windows":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-windows-amd64.zip"
    elif system == "Linux":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"
    else:
        raise Exception(f"{system} is not supported")
    download_path = _download_file(url)
    with zipfile.ZipFile(download_path, "r") as zip_ref:
        zip_ref.extractall(ngrok_path)


def _download_file(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    download_path = str(Path(tempfile.gettempdir(), local_filename))
    with open(download_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)
    return download_path


def start_ngrok(port):
    ngrok_address = _run_ngrok(port)
    print(f" * Running on {ngrok_address}")
    print(f" * Traffic stats available on http://127.0.0.1:4040")


def run_with_ngrok(app):
    """
    The provided Flask app will be securely exposed to the public internet via ngrok when run,
    and the its ngrok address will be printed to stdout
    :param app: a Flask application object
    :return: None
    """
    old_run = app.run

    def new_run(*args, **kwargs):
        port = kwargs.get('port', 5000)
        thread = Timer(1, start_ngrok, args=(port,))
        thread.setDaemon(True)
        thread.start()
        old_run(*args, **kwargs)
    app.run = new_run

In [0]:
#from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"
  
#app.run()

In [7]:

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
DIR_BASE = '/content/gdrive/My Drive/App/machine_learning/cv/shanghaidianqi_line5'
#DIR_DATASET = '20181117_03_08_1750_G1380.MP4'
DIR_DATASET = 'night.mp4'


In [9]:
!rm pantograph -rf
!git clone 'https://github.com/mashuai191/pantograph.git'

Cloning into 'pantograph'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 87 (delta 26), reused 60 (delta 10), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [0]:
#%cd '/content/pantograph/localization_yolo3'

In [0]:
!cp '/content/gdrive/My Drive/App/machine_learning/cv/shanghaidianqi_line5/yolo3/qingsong/yolo.h5' /content/pantograph/localization_yolo3/model_data

In [14]:
import sys
sys.path.append('/content/pantograph/localization_yolo3')

from yolo import YOLO

yolo = YOLO()


/content/pantograph/localization_yolo3/model_data/yolo.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:

#%cd /content/

In [16]:
!rm /content/flask-video-streaming -rf
!git clone https://github.com/mashuai191/flask-video-streaming.git

Cloning into 'flask-video-streaming'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 95 (delta 4), reused 4 (delta 1), pack-reused 79
Unpacking objects: 100% (95/95), done.


In [17]:
%cd /content/flask-video-streaming/

/content/flask-video-streaming


In [0]:
import os
import cv2
import numpy as np


from PIL import Image, ImageFont, ImageDraw

from base_camera import BaseCamera


class locCamera(BaseCamera):
    video_source = 0

    def __init__(self):
        if os.environ.get('OPENCV_CAMERA_SOURCE'):
            locCamera.set_video_source(int(os.environ['OPENCV_CAMERA_SOURCE']))
        super(locCamera, self).__init__()

    @staticmethod
    def set_video_source(source):
        locCamera.video_source = source

    @staticmethod
    def frames():
        #camera = cv2.VideoCapture(Camera.video_source)
        camera = cv2.VideoCapture(os.path.join(DIR_BASE, DIR_DATASET))
        if not camera.isOpened():
            raise RuntimeError('Could not start camera.')

        while True:
            # read current frame
            _, img = camera.read()

            # = img[200:625, 200:1150]
            print (img.shape)
            
            # let's predict
            image = Image.fromarray(img)
            image = yolo.detect_image(image)
            result = np.asarray(image)

            #v = Visualizer(cropped_image[:, :, ::-1], metadata=MetadataCatalog.get("balloon_" + 'test'), scale=1.2)
            #v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            #img_tmp = v.get_image()[:, :, ::-1]
            #cv2_imshow(v.get_image()[:, :, ::-1])


            imS = cv2.resize(result, (350, 250))
            #print (imS.shape)

            # encode as a jpeg image and return it
            yield cv2.imencode('.jpg', imS)[1].tobytes()


In [19]:
#!/usr/bin/env python
from importlib import import_module
import os
from flask import Flask, render_template, Response

# import camera driver
if os.environ.get('CAMERA'):
    Camera = import_module('camera_' + os.environ['CAMERA']).Camera
#else:
#    from camera import Camera

# Raspberry Pi camera module (requires picamera package)
# from camera_pi import Camera

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run


@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')


def gen(camera):
    """Video streaming generator function."""
    while True:
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    print ('video_feed pid {}', os.getpid())
    return Response(gen(locCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed1')
def video_feed1():
    """Video streaming route. Put this in the src attribute of an img tag."""
    print ('video_feed1, pid {}', os.getpid())
    return Response(gen(locCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

#if __name__ == '__main__':
#    app.run(host='0.0.0.0', threaded=True)
#from flask.flask_script import Manager

#manager = Manager(app)

#if __name__ == "__main__":
#    manager.run(host='0.0.0.0', threaded=True)
app.run(host='0.0.0.0', threaded=False, processes=1)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


 * Running on http://ea5f466c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Apr/2020 13:05:03] "GET / HTTP/1.1" 200 -


video_feed pid {} 618
Camera init.
Starting camera thread.pid 
 618 tid  140618880927616
(1536, 2048, 3)
************
 (416, 416, 3)


127.0.0.1 - - [02/Apr/2020 13:05:07] "GET /video_feed HTTP/1.1" 200 -


Found 0 boxes for img
3.00689387500006
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.1348084009998729
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.133881669999937
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.13500442100007604
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.1374549089998709
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.13317297799994776
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.13072424400002092
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.13049032800017812
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.13186239600008776
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.131208210000068
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.13082710799994857
(1536, 2048, 3)
************
 (416, 416, 3)
Found 0 boxes for img
0.13717728999995416
(1536, 2048, 3)
*****

In [0]:
import keras
print (keras.__version__)

In [0]:
!ls /usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf

In [0]:
! pip list | grep -i pil

In [0]:
!cat /etc/issue